In [1]:
import numpy as np
import pandas as pd

df = pd.read_excel(r"C:\Users\naras\OneDrive\Documents\Desktop\Weather Prediction usisng HMM\historical_weather_Vijayawada_data.xlsx")
df.head()

,date,precipitation,temp_max_celcius,temp_min_celcius,wind,weather_condition
0,2024-01-01,0.00,28.5,18.1,10.1,fog
1,2024-01-02,0.01,28.4,18.7,13.0,fog
2,2024-01-03,0.00,28.7,18.4,15.1,fog
3,2024-01-04,0.00,28.5,18.3,15.1,sun
4,2024-01-05,0.00,28.0,18.8,20.2,sun


In [2]:
df.shape
df.groupby('weather_condition').count()

,date,precipitation,temp_max_celcius,temp_min_celcius,wind
weather_condition,,,,,
fog,21,21,21,21,21
rain,139,139,139,139,139
sun,131,131,131,131,131


In [3]:
def get_precipitation_range_by_condition(df, weather_condition):
    condition_df = df.loc[df['weather_condition'] == weather_condition]
    if not condition_df.empty:
        max_precipitation = condition_df['precipitation'].max()
        min_precipitation = condition_df['precipitation'].min()
        return max_precipitation, min_precipitation
    else:
        return None, None

In [4]:
print("Precipitation:")
print("Sun:(max,min) =",get_precipitation_range_by_condition(df, 'sun'))
print("Fog:(max,min) =",get_precipitation_range_by_condition(df, 'fog'))
print("Rain:(max,min) =",get_precipitation_range_by_condition(df, 'rain'))

Precipitation:
Sun:(max,min) = (4.52, 0.0)
Fog:(max,min) = (0.66, 0.0)
Rain:(max,min) = (44.84, 0.01)


In [5]:
#one hot encoder weather
m = len(df['weather_condition'])
encode_weather = np.zeros(m)
for i in range(m):
    weather = -1
    if df['weather_condition'][i] == 'fog':
        weather = 0
    if df['weather_condition'][i] == 'rain':
        weather = 1
    if df['weather_condition'][i] == 'sun':
        weather = 2
    encode_weather[i] = weather
df['Encode Weather'] = encode_weather
df

,date,precipitation,temp_max_celcius,temp_min_celcius,wind,weather_condition,Encode Weather
0,2024-01-01,0.00,28.5,18.1,10.1,fog,0.0
1,2024-01-02,0.01,28.4,18.7,13.0,fog,0.0
2,2024-01-03,0.00,28.7,18.4,15.1,fog,0.0
3,2024-01-04,0.00,28.5,18.3,15.1,sun,2.0
4,2024-01-05,0.00,28.0,18.8,20.2,sun,2.0
...,...,...,...,...,...,...,...
286,2024-10-13,4.25,31.2,23.9,10.4,rain,1.0
287,2024-10-14,3.54,30.7,23.8,9.4,rain,1.0
288,2024-10-15,6.16,30.3,23.9,10.1,rain,1.0
289,2024-10-16,8.35,29.5,23.9,20.9,rain,1.0


In [6]:
from datetime import datetime
import numpy as np

arr_date = df['date']
season = np.zeros(df.shape[0])
duration = len(arr_date)

for i in range(duration):
    date = datetime.strptime(arr_date[i], '%Y-%m-%d')
    month = date.month
    if month == 11 or month == 12 or month == 1 or month == 2:
        season[i] = 0  # Winter
    elif month >= 3 and month <= 5:
        season[i] = 1  # Summer
    elif month >= 6 and month <= 9:
        season[i] = 2  # Rainy (Monsoon)
    elif month == 10:
        season[i] = 3  # Transition period to winter

df['Encode Season'] = season

df


,date,precipitation,temp_max_celcius,temp_min_celcius,wind,weather_condition,Encode Weather,Encode Season
0,2024-01-01,0.00,28.5,18.1,10.1,fog,0.0,0.0
1,2024-01-02,0.01,28.4,18.7,13.0,fog,0.0,0.0
2,2024-01-03,0.00,28.7,18.4,15.1,fog,0.0,0.0
3,2024-01-04,0.00,28.5,18.3,15.1,sun,2.0,0.0
4,2024-01-05,0.00,28.0,18.8,20.2,sun,2.0,0.0
...,...,...,...,...,...,...,...,...
286,2024-10-13,4.25,31.2,23.9,10.4,rain,1.0,3.0
287,2024-10-14,3.54,30.7,23.8,9.4,rain,1.0,3.0
288,2024-10-15,6.16,30.3,23.9,10.1,rain,1.0,3.0
289,2024-10-16,8.35,29.5,23.9,20.9,rain,1.0,3.0


In [7]:
max_temp = df['temp_min_celcius'].max()
min_temp = df['temp_min_celcius'].min()
max_wind = df['wind'].max()
min_wind = df['wind'].min()
print("max of temp_min:", max_temp)
print("min of temp_min:", min_temp)
print("max of wind:", max_wind)
print("min of wind:", min_wind)

max of temp_min: 33.3
min of temp_min: 16.6
max of wind: 36.4
min of wind: 6.1


In [8]:
#We will see how many days it is snowy when the degree is above zero
#And how many day it is rainy when the degree is below zero
count_rain_below_zero = 0
count_rain = 0

m = df.shape[0]

In [9]:
for i in range(m):
    if df['weather_condition'][i] == 'rain':
        count_rain += 1
        if df['temp_min_celcius'][i] < 0.0:  # Check if the minimum temperature is below zero
            count_rain_below_zero += 1
            print("Min temp when it rains:", df['temp_min_celcius'][i])

print("Total rainy days:", count_rain)
print("Rainy days with min temp below zero:", count_rain_below_zero)

Total rainy days: 139
Rainy days with min temp below zero: 0


In [10]:
m = df.shape[0]
encode_temp = np.zeros(m)
count_very_cold = 0
count_cold = 0
count_cool = 0
count_warm = 0

In [11]:
for i in range(m):
    if df['temp_min_celcius'][i] <= 18.0:
        encode_temp[i] = 0
        count_very_cold += 1
    if df['temp_min_celcius'][i] > 18.0 and df['temp_min_celcius'][i] <=24.0:
        encode_temp[i] = 1
        count_cold += 1
    if df['temp_min_celcius'][i] > 24.0 and df['temp_min_celcius'][i] <= 30.0:
        encode_temp[i] = 2
        count_cool += 1
    if df['temp_min_celcius'][i] > 30:
        encode_temp[i] = 3
        count_warm += 1

In [12]:
df['Encode Temp'] = encode_temp
print(count_very_cold)
print(count_cold)
print(count_cool)
print(count_warm)

10
93
180
8


In [13]:
import numpy as np

m = df.shape[0]
encode_pre = np.zeros(m)

# Precipitation classification counters
none_pre = 0
light_pre = 0
moderate_pre = 0
heavy_pre = 0

for i in range(m):
    pre_value = df['precipitation'][i]
    if pre_value == 0.0:
        encode_pre[i] = 0  # No Rain
        none_pre += 1
    elif 0 < pre_value <= 10:
        encode_pre[i] = 1  # Light Rain
        light_pre += 1
    elif 10 < pre_value <= 30:
        encode_pre[i] = 2  # Moderate Rain
        moderate_pre += 1
    else:
        encode_pre[i] = 3  # Heavy Rain
        heavy_pre += 1

df['Encode Precipitation'] = encode_pre

# Print counts for each category
print("No Rain:", none_pre)
print("Light Rain:", light_pre)
print("Moderate Rain:", moderate_pre)
print("Heavy Rain:", heavy_pre)

No Rain: 121
Light Rain: 155
Moderate Rain: 14
Heavy Rain: 1


In [14]:
encode_wind = np.zeros(m)
wind_1 = 0
wind_2 = 0
wind_3 = 0
wind_4 = 0
for i in range(m):
    wind_value = df['wind'][i]
    if wind_value < 12.0:
        encode_wind[i] = 0
        wind_1 += 1
    if wind_value >= 12.0 and wind_value < 24.0:
        encode_wind[i] = 1
        wind_2 += 1
    if wind_value >= 24.0 and wind_value < 30.0:
        encode_wind[i] = 2
        wind_3 += 1
    if wind_value >= 30.0:
        encode_wind[i] = 3
        wind_4 += 1
df['Encode Wind'] = encode_wind
print(wind_1)
print(wind_2)
print(wind_3)
print(wind_4)

28
167
87
9


In [15]:
df.head()

,date,precipitation,temp_max_celcius,temp_min_celcius,wind,weather_condition,Encode Weather,Encode Season,Encode Temp,Encode Precipitation,Encode Wind
0,2024-01-01,0.00,28.5,18.1,10.1,fog,0.0,0.0,1.0,0.0,0.0
1,2024-01-02,0.01,28.4,18.7,13.0,fog,0.0,0.0,1.0,1.0,1.0
2,2024-01-03,0.00,28.7,18.4,15.1,fog,0.0,0.0,1.0,0.0,1.0
3,2024-01-04,0.00,28.5,18.3,15.1,sun,2.0,0.0,1.0,0.0,1.0
4,2024-01-05,0.00,28.0,18.8,20.2,sun,2.0,0.0,1.0,0.0,1.0


In [16]:
import numpy as np
import pandas as pd

# Zigzag splitting of data
def zigzag_split(df, train_fraction=0.8):
    # Create alternating indices for train/test split
    indices = np.arange(df.shape[0])
    np.random.shuffle(indices)  # Shuffle to ensure randomness
    
    train_size = int(len(indices) * train_fraction)
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    
    train_data = df.iloc[train_indices]
    test_data = df.iloc[test_indices]
    
    return train_data, test_data

# Augment and extract features
def augment_features(df):
    return pd.DataFrame({
        'Season': df['Encode Season'],
        'Temp': df['Encode Temp'],
        'Precipitation': df['Encode Precipitation'],
        'Wind': df['Encode Wind'],
        'Weather': df['Encode Weather']
    })

def extract_features(df):
    return np.column_stack((df['Season'], df['Temp'], df['Precipitation'], df['Wind'], df['Weather']))

# Perform zigzag split
train_data, test_data = zigzag_split(df)

# Extract features from train data
features = extract_features(augment_features(train_data))
print(features)


[[1. 1. 0. 2. 2.]
 [2. 2. 1. 1. 1.]
 [2. 2. 1. 3. 1.]
 ...
 [0. 1. 0. 1. 0.]
 [1. 1. 0. 2. 2.]
 [2. 1. 1. 1. 1.]]


In [17]:
print(extract_features(augment_features(test_data)))

[[1. 2. 1. 1. 2.]
 [2. 2. 0. 2. 2.]
 [2. 2. 1. 1. 2.]
 [2. 2. 1. 2. 1.]
 [2. 2. 1. 1. 1.]
 [2. 2. 1. 2. 1.]
 [0. 1. 1. 1. 0.]
 [3. 1. 1. 1. 1.]
 [0. 1. 0. 1. 2.]
 [0. 1. 0. 1. 0.]
 [2. 2. 1. 0. 1.]
 [0. 0. 0. 1. 2.]
 [2. 2. 1. 2. 2.]
 [2. 2. 0. 1. 2.]
 [2. 2. 1. 1. 2.]
 [0. 1. 0. 2. 0.]
 [2. 2. 1. 1. 1.]
 [2. 2. 1. 1. 1.]
 [1. 2. 0. 1. 2.]
 [1. 2. 0. 1. 2.]
 [1. 2. 0. 2. 2.]
 [2. 1. 2. 2. 1.]
 [2. 2. 1. 1. 2.]
 [0. 1. 1. 1. 1.]
 [1. 2. 1. 0. 1.]
 [2. 2. 1. 1. 1.]
 [2. 2. 2. 1. 1.]
 [1. 2. 1. 2. 2.]
 [0. 0. 0. 1. 2.]
 [1. 2. 1. 3. 1.]
 [0. 0. 0. 1. 2.]
 [1. 1. 0. 2. 2.]
 [3. 1. 1. 1. 1.]
 [2. 2. 0. 1. 2.]
 [2. 1. 1. 1. 1.]
 [1. 2. 0. 2. 2.]
 [0. 0. 0. 1. 2.]
 [2. 2. 1. 0. 1.]
 [1. 2. 1. 0. 2.]
 [1. 2. 0. 2. 2.]
 [2. 3. 0. 1. 2.]
 [2. 2. 2. 0. 1.]
 [1. 3. 1. 2. 2.]
 [2. 2. 1. 1. 1.]
 [2. 2. 1. 3. 1.]
 [2. 2. 1. 1. 1.]
 [0. 1. 0. 1. 2.]
 [1. 1. 0. 2. 2.]
 [2. 2. 1. 2. 1.]
 [2. 2. 1. 2. 2.]
 [1. 2. 1. 2. 2.]
 [2. 2. 1. 2. 1.]
 [2. 2. 2. 0. 1.]
 [1. 2. 0. 2. 2.]
 [1. 2. 1. 1. 1.]
 [2. 2. 1.

In [18]:
!pip install hmmlearn


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from hmmlearn.hmm import GaussianHMM
model = GaussianHMM(n_components = 3,random_state = 43)  # it will be 3 only beacuse of Fog,Rain,Sun
model.fit(features)
test_data.reset_index(inplace=True, drop=True)
test_data.shape


C:\Users\naras\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\naras\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\naras\.conda\envs\narsimha\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\naras\.conda\envs\narsimha\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\naras\.conda\envs\narsimha\Lib\subprocess.py"

(59, 11)

In [20]:
num_previous_days = 2
test_data['Encode Precipitation'].unique()

array([1., 0., 2.])

In [21]:
from tqdm import tqdm

sample_weather = np.array([0.0, 1.0, 2.0])
test_size = test_data.shape[0]
predicted_weather = []
for i in tqdm(range(test_size)):
    previous_days_start_index = max(0, i-num_previous_days)
    previous_days_end_index = max(0, i)
    previous_days = extract_features(augment_features(test_data.iloc[previous_days_start_index:previous_days_end_index]))
    
    likelihood_scores = []
    for weather in sample_weather:
        current_day = [test_data['Encode Season'][i], test_data['Encode Temp'][i], test_data['Encode Precipitation'][i], test_data['Encode Wind'][i],weather]
        sequence = np.row_stack((previous_days, current_day))
        likelihood_scores.append(model.score(sequence))
    most_probable_weather = sample_weather[np.argmax(likelihood_scores)]
    predicted_weather.append(most_probable_weather)

print(predicted_weather)

100%|█████████████████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 898.59it/s]

[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 2.0]


In [22]:
count = 0
for i in range(test_size):
    if test_data['Encode Weather'][i] == predicted_weather[i]:
        count += 1
print("accuracy:", float(count/test_size))

accuracy: 0.7627118644067796


In [23]:
def score(predicted_weather):
    count = 0
    for i in range(test_size):
        if test_data['Encode Weather'][i] == predicted_weather[i]:
            count += 1
    return float(count/test_size)
import matplotlib.pyplot as plt

n_component_values = [3] # it will be 3 only beacuse of Fog,Rain,Sun
arr_previous_days = [1,2,3,4,5,6,7]

for n in n_component_values:
    tmp_model = GaussianHMM(n_components = n, random_state = 43)
    tmp_model.fit(features)
    for tmp_num_previous_days in arr_previous_days: 
        sample_weather = np.array([0.0, 1.0, 2.0])
        test_size = test_data.shape[0]
        tmp_predicted_weather = []
        for i in range(test_size):
            previous_days_start_index = max(0, i-tmp_num_previous_days)
            previous_days_end_index = max(0, i)
            previous_days = extract_features(augment_features(test_data.iloc[previous_days_start_index:previous_days_end_index]))
    
            likelihood_scores = []
            for weather in sample_weather:
                current_day = [test_data['Encode Season'][i], test_data['Encode Temp'][i], test_data['Encode Precipitation'][i], test_data['Encode Wind'][i],weather]
                sequence = np.row_stack((previous_days, current_day))
                likelihood_scores.append(tmp_model.score(sequence))
            most_probable_weather = sample_weather[np.argmax(likelihood_scores)]
            tmp_predicted_weather.append(most_probable_weather)
        print("(n_components, num_previous_days, score)", n, tmp_num_previous_days,score(tmp_predicted_weather))

(n_components, num_previous_days, score) 3 1 0.7627118644067796
(n_components, num_previous_days, score) 3 2 0.7627118644067796
(n_components, num_previous_days, score) 3 3 0.7627118644067796
(n_components, num_previous_days, score) 3 4 0.7627118644067796
(n_components, num_previous_days, score) 3 5 0.7627118644067796
(n_components, num_previous_days, score) 3 6 0.7627118644067796
(n_components, num_previous_days, score) 3 7 0.7627118644067796


In [24]:
import joblib

# Save your model
joblib.dump(model, 'weather_model.pkl')


['weather_model.pkl']

In [25]:
# Load your model
model = joblib.load('weather_model.pkl')

In [26]:
from tqdm import tqdm

sample_weather = np.linspace(0.0,4.0,5)
test_size = test_data.shape[0]
predicted_weather = []
for i in tqdm(range(test_size)):
    previous_days_start_index = max(0, i-num_previous_days)
    previous_days_end_index = max(0, i)
    previous_days = extract_features(augment_features(test_data.iloc[previous_days_start_index:previous_days_end_index]))
    
    likelihood_scores = []
    for weather in sample_weather:
        current_day = [test_data['Encode Season'][i], test_data['Encode Temp'][i], test_data['Encode Precipitation'][i], test_data['Encode Wind'][i],weather]
        sequence = np.row_stack((previous_days, current_day))
        likelihood_scores.append(model.score(sequence))
    most_probable_weather = sample_weather[np.argmax(likelihood_scores)]
    predicted_weather.append(most_probable_weather)

print(predicted_weather)

100%|█████████████████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 295.21it/s]

[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 2.0]


In [27]:
count = 0
for i in range(test_size):
    if test_data['Encode Weather'][i] == predicted_weather[i]:
        count += 1
print("accuracy:", float(count/test_size))

accuracy: 0.7627118644067796


In [28]:
import numpy as np
import pandas as pd
from datetime import datetime
from hmmlearn.hmm import GaussianHMM
import joblib



In [29]:
# Get user input for weather features
user_input = input("Enter the weather features as 'Season, Temp, Precipitation, Wind': ")
season_encoded, temp_encoded, precipitation_encoded, wind_encoded = map(float, user_input.split(','))

# Validate the season input
if season_encoded not in [0, 1, 2, 3]:
    print("Invalid season. Please enter 0 for Winter, 1 for Summer, 2 for Rainy, 3 for Transition.")
    exit()

# Prepare the feature array for prediction
feature_array = np.array([[season_encoded, temp_encoded, precipitation_encoded, wind_encoded, -1]])  # -1 for unknown weather

# Use the model to predict the weather
predicted_weather = model.predict(feature_array)

# Define possible weather conditions
weather_conditions = {0: 'fog', 1: 'rain', 2: 'sun'}

# Get the predicted condition
if predicted_weather[0] in weather_conditions:
    predicted_condition = weather_conditions[predicted_weather[0]]
else:
    predicted_condition = 'unknown'  # Assign 'unknown' if the prediction is not in the defined conditions

print("Predicted weather condition for the input:", predicted_condition)


Enter the weather features as 'Season, Temp, Precipitation, Wind':  0,1,0,0


Predicted weather condition for the input: fog
